In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm

# 1. データの準備（Sand_03 を想定）
# df は全データ。CCU 100以上を「1」、それ未満を「0」とするフラグを作成
df['selection_flag'] = (df['peak_ccu'] >= 100).astype(int)

# 2. 第1段階：プロビット回帰（選択方程式）
# ここには「生存（100突破）」に効くが「売上（効果）」には直接関係なさそうな変数（Z）を入れる
# 例: 早期アクセスかどうか(is_early_access)、言語数(language_count)など
Z = df[['is_early_access', 'language_count', 'genre_indie']] # 排外制約の候補
Z = sm.add_constant(Z)
probit_model = sm.Probit(df['selection_flag'], Z).fit()

# 3. 逆ミルズ比 (IMR) の計算
# 各データに対して、個別に「運の偏り」を算出
phi = norm.pdf(probit_model.fittedvalues)  # 標準正規分布の密度
Phi = norm.cdf(probit_model.fittedvalues)  # 標準正規分布の累積分布
df['imr'] = phi / Phi

# 4. 第2段階：OLS回帰（結果方程式）
# CCU 100以上のゲーム「だけ」を取り出す
df_sub = df[df['selection_flag'] == 1].copy()

# ここで discount の効果を測る。必ず IMR をコントロール変数として入れる
X = df_sub[['discount_rate', 'price', 'metascore', 'imr']] # imrが掃除機の役割を果たす
X = sm.add_constant(X)
y = np.log(df_sub['peak_ccu'] + 1) # CCUは冪乗分布なので log をとるのが定石

heckit_results = sm.OLS(y, X).fit()

# 5. 結果の確認
print(heckit_results.summary())